Objective: Build a Python-based prototype that fuses simulated radar and infrared sensor data to track a moving object (e.g., a drone) and visualize its trajectory on a 2D plot, demonstrating basic situational awareness for an Air Force scenario.

In [ ]:
# install libraries
!pip install -q numpy pandas matplotlib filterpy

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import filterpy.kalman

Step 1: Generate Synthetic Data

Goal: Create a dataset simulating a drone’s true 2D path and noisy sensor measurements (radar, infrared).

In [ ]:
# define drone path
# simulate drone moving in straight line

# 0 to 100 seconds
t = np.arange(0, 101, 1)

# true positions
# x axis = t, y axis = 0.5t
true_x = t
true_y = 0.5 * t
true_positions = np.column_stack((true_x, true_y))

In [ ]:
# noisy sensor data
# radar = 0.5
# infrared = 0.3 (less noisy than radar)

np.random.seed(42)  # for reproducibility

# radar
radar_x = true_x + np.random.normal(0, 0.5, len(t))
radar_y = true_y + np.random.normal(0, 0.5, len(t))
radar_measurements = np.column_stack((radar_x, radar_y))

# infrared
ir_x = true_x + np.random.normal(0, 0.3, len(t))
ir_y = true_y + np.random.normal(0, 0.3, len(t))
ir_measurements = np.column_stack((ir_x, ir_y))

In [ ]:
# save to df
data = pd.DataFrame({
    'time': t,
    'true_x': true_x,
    'true_y': true_y,
    'radar_x': radar_x,
    'radar_y': radar_y,
    'ir_x': ir_x,
    'ir_y': ir_y
})

# save for reference
data.to_csv('sensor_data.csv', index=False)

In [ ]:
# test data
# expected output: a plot with a blue line (true path), scattered red dots (radar), and green dots (ir) around it. If dots are too far from the line, adjust noise levels

plt.scatter(data['radar_x'], data['radar_y'], c='red', label='Radar', alpha=0.5)
plt.scatter(data['ir_x'], data['ir_y'], c='green', label='IR', alpha=0.5)

plt.plot(data['true_x'], data['true_y'], 'b-', label='True Path')

plt.xlabel('X Position')
plt.ylabel('Y Position')

plt.title('Raw Sensor Data vs True Path')

plt.legend()
plt.grid(True)
plt.show()

Step 2: Implement Data Fusion with Kalman Filter

Goal: Use a Kalman Filter to fuse radar and IR measurements and estimate the drone’s true position.

In [ ]:
# set up kalman filter:
#   state: [x, y, vx, vy] (position and velocity in x, y).
#   measurements: [x, y] from radar or ir.

from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise

# kalman filter
kf = KalmanFilter(dim_x=4, dim_z=2)  # 4 state variables, 2 measurements

# state transition matrix (constant velocity model)
dt = 1.0  # time step
kf.F = np.array([[1, 0, dt, 0],
                 [0, 1, 0, dt],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])

# measurement matrix (observe x, y)
kf.H = np.array([[1, 0, 0, 0],
                 [0, 1, 0, 0]])

# initial state (start at first true position, zero velocity)
kf.x = np.array([true_x[0], true_y[0], 0, 0])

# state covariance (initial uncertainty)
kf.P *= 10

# measurement noise covariance (combine radar and IR noise)
kf.R = np.array([[0.4, 0], [0, 0.4]])  # average of radar (0.5) and IR (0.3)

# process noise
kf.Q = Q_discrete_white_noise(dim=2, dt=dt, var=0.01, block_size=2)

In [ ]:
# fuse measurements
fused_positions = []

for i in range(len(t)):
    # predict step
    kf.predict()
    # update with averaged measurements
    z = (radar_measurements[i] + ir_measurements[i]) / 2
    kf.update(z)
    # save fused position
    fused_positions.append(kf.x[:2])

fused_positions = np.array(fused_positions)

# add to df
data['fused_x'] = fused_positions[:, 0]
data['fused_y'] = fused_positions[:, 1]

In [ ]:
# test fusion
# expected output: black dashed line (fused path) should be closer to the blue line (true path) than the scattered dots.
#   if the fused path is erratic, tweak kf.R or kf.Q (e.g., increase measurement noise to 0.5 or reduce process noise to 0.005).

plt.scatter(data['radar_x'], data['radar_y'], c='red', label='Radar', alpha=0.3)
plt.scatter(data['ir_x'], data['ir_y'], c='green', label='IR', alpha=0.3)

plt.plot(data['true_x'], data['true_y'], 'b-', label='True Path')
plt.plot(data['fused_x'], data['fused_y'], 'k--', label='Fused Path')

plt.xlabel('X Position')
plt.ylabel('Y Position')

plt.title('Fused Trajectory vs True Path')

plt.legend()
plt.grid(True)
plt.show()

Step 3: Evaluate Results

Goal: Quantify fusion performance using Mean Squared Error (MSE).

In [ ]:
# compute MSE: compare true positions to radar, ir, and fused positions
# expected output: MSE for fused positions should be lower than for radar or ir (e.g., ~0.1 vs. ~0.25 for radar, ~0.09 for IR)

from sklearn.metrics import mean_squared_error

mse_radar = mean_squared_error(data[['true_x', 'true_y']], data[['radar_x', 'radar_y']])
mse_ir = mean_squared_error(data[['true_x', 'true_y']], data[['ir_x', 'ir_y']])
mse_fused = mean_squared_error(data[['true_x', 'true_y']], data[['fused_x', 'fused_y']])

print(f"MSE Radar: {mse_radar:.4f}")
print(f"MSE IR: {mse_ir:.4f}")
print(f"MSE Fused: {mse_fused:.4f}")

Test Robustness:
Add more noise to one sensor (e.g., radar std=1.0) and rerun. The fused path should still track reasonably well.

Step 4: Visualize Results

Goal: Create a polished situational awareness plot.

In [ ]:
plt.figure(figsize=(8, 6))

plt.scatter(data['radar_x'], data['radar_y'], c='red', label='Radar', alpha=0.3, s=20)
plt.scatter(data['ir_x'], data['ir_y'], c='green', label='IR', alpha=0.3, s=20)

plt.plot(data['true_x'], data['true_y'], 'b-', label='True Path', linewidth=2)
plt.plot(data['fused_x'], data['fused_y'], 'k--', label='Fused Path', linewidth=2)

plt.xlabel('X Position (m)')
plt.ylabel('Y Position (m)')

plt.title('Drone Tracking with Sensor Fusion')

plt.legend()
plt.grid(True)
plt.savefig('tracking_result.png', dpi=300)
plt.show()